LIb



In [2]:
!pip install easyocr opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 19.2 MB/s eta 0:00:00


In [3]:
!pip install ultralytics easyocr opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.7 MB/s eta 0:00:00


In [15]:
from ultralytics import YOLO
import cv2
import easyocr
import re

# -----------------------------
# CONFIG
# -----------------------------
MODEL_PATH = '/content/drive/MyDrive/number_plate_detection/runs/detect/train3/weights/best.pt'
IMAGE_PATH = '/content/drive/MyDrive/number_plate_detection/vid-3/frame-520.jpg'

MIN_PLATE_HEIGHT = 40
MIN_PLATE_WIDTH = 120

INDIAN_PLATE_REGEX = r'^[A-Z]{2}[0-9]{2}[A-Z]{1,2}[0-9]{4}$'

# -----------------------------
# LOAD MODELS
# -----------------------------
model = YOLO(MODEL_PATH)
reader = easyocr.Reader(['en'], gpu=False)

# -----------------------------
# HELPER FUNCTIONS
# -----------------------------
def clean_plate_text(text):
    text = text.upper()
    text = re.sub(r'[^A-Z0-9]', '', text)
    return text

def normalize_common_ocr_confusions(text):
    """
    Generic OCR normalization valid for Indian plates
    """
    replacements = {
        'O': '0',
        'I': '1',
        'Z': '2',
        'S': '5'
    }
    for k, v in replacements.items():
        text = text.replace(k, v)
    return text

def is_valid_indian_plate(text):
    return bool(re.match(INDIAN_PLATE_REGEX, text))

# -----------------------------
# RUN DETECTION
# -----------------------------
results = model(IMAGE_PATH, conf=0.25)

img = cv2.imread(IMAGE_PATH)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

final_results = []

for r in results:
    for box in r.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        plate = img_rgb[y1:y2, x1:x2]
        h, w = plate.shape[:2]

        if h < MIN_PLATE_HEIGHT or w < MIN_PLATE_WIDTH:
            continue

        ocr_result = reader.readtext(
            plate,
            allowlist='ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789',
            detail=1
        )

        for detection in ocr_result:
            raw_text = detection[1]
            ocr_conf = detection[2]

            cleaned = clean_plate_text(raw_text)
            normalized = normalize_common_ocr_confusions(cleaned)
            valid = is_valid_indian_plate(normalized)

            final_results.append({
                "raw_text": raw_text,
                "normalized_text": normalized,
                "ocr_confidence": float(ocr_conf),
                "valid_indian_plate": valid,
                "bbox": [x1, y1, x2, y2]
            })

# -----------------------------
# OUTPUT
# -----------------------------
print("FINAL RESULTS:")
for res in final_results:
    print(res)

print("\nTotal OCR detections:", len(final_results))



image 1/1 /content/drive/MyDrive/number_plate_detection/vid-3/frame-520.jpg: 384x640 2 number_plates, 216.0ms
Speed: 7.6ms preprocess, 216.0ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
FINAL RESULTS:
{'raw_text': 'THO7CU94571', 'normalized_text': 'TH07CU94571', 'ocr_confidence': 0.5790295136736382, 'valid_indian_plate': False, 'bbox': [543, 821, 713, 886]}

Total OCR detections: 1
